In the following model, we use optimal control theory to determine the buy and sell bounds for a strategy based on cointegration, long-short, and long-only approaches.

The model is taken from the book "Algorithmic and High-Frequency Trading" by Alvaro Cartea, Sebastian Jaimungal, and José Penalva.

#### Let's define the most important equations of the model: :

$$ d\epsilon_t = \kappa (\theta - \epsilon_t) dt + \sigma dW_t $$
$$ F_+ (\epsilon) = \int_{0}^{\infty} u^{\rho - 1} e^{-\sqrt{\frac{2\kappa}{\sigma^2}} (\epsilon - \theta) u - \frac{1}{2} u^2} du $$
$$ F_- (\epsilon) = \int_{0}^{\infty} u^{\rho - 1} e^{\sqrt{\frac{2\kappa}{\sigma^2}} (\epsilon - \theta) u - \frac{1}{2} u^2} du $$
$$ F'_+ (\epsilon) = \int_{0}^{\infty} u^{\rho} e^{-\sqrt{\frac{2\kappa}{\sigma^2}} (\epsilon - \theta) u - \frac{1}{2} u^2} du $$
$$ F'_- (\epsilon) = \int_{0}^{\infty} u^{\rho} e^{\sqrt{\frac{2\kappa}{\sigma^2}} (\epsilon - \theta) u - \frac{1}{2} u^2} du $$
$$ H_+ (\epsilon) = \frac{F_+ (\epsilon)}{F'_+ (\epsilon^*_-)} (\epsilon^*_+ - c) \mathbf{1}_{\epsilon < \epsilon^*_+} + (\epsilon - c) \mathbf{1}_{\epsilon \geq \epsilon^*_-} $$
$$ H'_+ (\epsilon) = \frac{F'_+ (\epsilon)}{F_+ (\epsilon^*_-)} (\epsilon^*_+ - c) \mathbf{1}_{\epsilon < \epsilon^*_+} + \mathbf{1}_{\epsilon \geq \epsilon^*_-} $$
$$ (\epsilon^* - c) F'_+ (\epsilon^+) = F_+ (\epsilon^*) $$
$$ (\epsilon^*_- + c) F'_- (\epsilon^-) = F_- (\epsilon^*_-) $$

### Parameters
- c : transaction cost
- theta : long-term mean parameter
- kappa : mean reversion speed
- sigma : volatility
- rho : discounting parameter

In [3]:
import numpy as np
from scipy.optimize import minimize
from scipy.integrate import quad
from scipy.optimize import fsolve, root
import warnings
from scipy.integrate import IntegrationWarning

warnings.filterwarnings("ignore", category=IntegrationWarning)

class CointBornes:
    def __init__(self, c, theta, kappa, sigma, rho):
        self.c = c
        self.theta = theta
        self.kappa = kappa
        self.sigma = sigma
        self.rho = rho

    def int_plus(self, u, epsilon):
        return u**(self.rho/self.kappa - 1) * np.exp(-np.sqrt(2*self.kappa/self.sigma**2)*(self.theta - epsilon)*u - 0.5*u**2)

    def int_minus(self, u, epsilon):
        return u**(self.rho/self.kappa - 1) * np.exp(np.sqrt(2*self.kappa/self.sigma**2)*(self.theta - epsilon)*u - 0.5*u**2)

    def dint_plus(self, u, epsilon):
        return u**(self.rho/self.kappa - 1) * np.sqrt(2*self.kappa / self.sigma**2) * u * np.exp(-np.sqrt(2*self.kappa/self.sigma**2)*(self.theta - epsilon)*u - 0.5*u**2)

    def dint_minus(self, u, epsilon):
        return u**(self.rho/self.kappa - 1) * (-np.sqrt(2 * self.kappa / self.sigma**2)) * u * np.exp(np.sqrt(2*self.kappa / self.sigma**2)*(self.theta - epsilon)*u - 0.5*u**2)

    def F_plus(self, epsilon):
        res, error = quad(self.int_plus, 0, np.inf, (epsilon,))
        return res

    def F_minus(self, epsilon):
        res, error = quad(self.int_minus, 0, np.inf, (epsilon,))
        return res

    def dF_plus(self, epsilon, analytic):
        if analytic:
            res, error = quad(self.dint_plus, 0, np.inf, (epsilon,))
            return res
        else:
            return (self.F_plus(epsilon+1e-6) - self.F_plus(epsilon)) / 1e-6
    def df_minus(self, epsilon, analytic):

        if analytic:
            res, error = quad(self.dint_minus, 0, np.inf, (epsilon,))
            return res
        else:
            return (self.F_minus(epsilon+1e-6) - self.F_minus(epsilon))/1e-6

    def H_plus(self, epsilon):
        return (self.F_plus(epsilon)/self.F_plus(self.epsilon_star)) * (self.epsilon_star - self.c) if epsilon < self.epsilon_star else epsilon - self.c

    def dH_plus(self, epsilon, analytic):

        if analytic:
            return (self.epsilon_star - self.c) * self.dF_plus(epsilon, analytic) / self.F_plus(self.epsilon_star) if epsilon < self.epsilon_star else 1
        else:
            return (self.H_plus(epsilon+1e-6) - self.H_plus(epsilon)) / 1e-6

    def solve_optimal_long_short(self, epsilon, analytic):
        return (epsilon + self.c) * self.df_minus(epsilon, analytic) - self.F_minus(epsilon)

    def solve_optimal_stop(self, epsilon_star, analytic):
        return (epsilon_star - self.c) * self.dF_plus(epsilon_star, analytic) - self.F_plus(epsilon_star)

    def solve_optimal_entry(self, epsilon_botstar, analytic):
        return (self.H_plus(epsilon_botstar) - epsilon_botstar - self.c) * self.df_minus(epsilon_botstar, analytic) - (self.dH_plus(epsilon_botstar, analytic) - 1) * self.F_minus(epsilon_botstar)

    def get_opti_params(self, analytic, long_only, init_low, init_high):
        if long_only:
            initial = init_high
            solution = root(self.solve_optimal_stop, initial, method='hybr', tol=0.000001, args=(analytic,))
            epsilon_star = solution.x[0]
            self.epsilon_star = epsilon_star
            initial = init_low
            solution = root(self.solve_optimal_entry, initial, method='hybr', tol=0.000001, args=(analytic,))
            epsilon_botstar = solution.x[0]

            return epsilon_botstar, epsilon_star

        else:
            initial = init_high
            solution = root(self.solve_optimal_stop, initial, method='hybr', tol=0.000001, args=(analytic,))
            epsilon_star_plus = solution.x[0]
            initial = init_low
            solution = root(self.solve_optimal_long_short, initial, method='hybr', tol=0.000001, args=(analytic,))
            epsilon_star_minus = solution.x[0]

            return float(epsilon_star_minus), float(epsilon_star_plus)




In [4]:
c = 0.01            # The higher the value, the more transaction costs are taken into account.
theta = 1           
kappa = 0.5
sigma = 0.5
rho = 0.01           # The value of rho depends on how much we believe that money in the future is worth less than money today.

test = CointBornes(c, theta, kappa, sigma, rho)
values = test.get_opti_params(analytic = True, long_only = False, init_high = 3, init_low = -2)
print(f"Entry bound: {values[0]:.4f}, Exit bound: {values[1]:.4f}")



Entry bound: -0.4060, Exit bound: 1.9537
